datafutures
@guzmanwolfrank:github

Fetches futures data on micro emini indexes like nq, es, ym. 

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Define tickers and their approximate inception dates (CME launch)
tickers_info = {
    'MNQ=F': '2019-05-06',  # Micro E-mini Nasdaq-100
    'MES=F': '2019-05-06',  # Micro E-mini S&P 500
    'MYM=F': '2020-05-18'   # Micro E-mini Dow
}

# Current date as end
end_date = datetime.now().strftime('%Y-%m-%d')

for ticker, start_date in tickers_info.items():
    print(f"\nFetching daily data for {ticker} from {start_date} to {end_date}...")
    data = yf.download(ticker, start=start_date, end=end_date, interval='1d', progress=False)
    
    if not data.empty:
        print(f"Shape: {data.shape} (rows: {len(data)}, columns: {list(data.columns)})")
        print(data.head())
        print(f"\nEarliest date: {data.index[0].date()}")
        print(f"Latest date: {data.index[-1].date()}")
        # Save to CSV with timestamp in filename
        filename = f"{ticker}_daily_historical_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        data.to_csv(filename)
        print(f"Saved to {filename}")
    else:
        print(f"No data available for {ticker}")


Fetching daily data for MNQ=F from 2019-05-06 to 2025-11-23...


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_24904\1500899964.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval='1d', progress=False)


Shape: (1652, 5) (rows: 1652, columns: [('Close', 'MNQ=F'), ('High', 'MNQ=F'), ('Low', 'MNQ=F'), ('Open', 'MNQ=F'), ('Volume', 'MNQ=F')])
Price         Close     High      Low     Open  Volume
Ticker        MNQ=F    MNQ=F    MNQ=F    MNQ=F   MNQ=F
Date                                                  
2019-05-06  7803.50  7865.00  7668.75  7865.00   91921
2019-05-07  7674.00  7804.00  7581.50  7745.25  159356
2019-05-08  7647.50  7697.75  7600.75  7668.50  174381
2019-05-09  7595.25  7644.25  7481.25  7640.00  181633
2019-05-10  7610.25  7653.50  7435.50  7592.00  165887

Earliest date: 2019-05-06
Latest date: 2025-11-21
Saved to MNQ=F_daily_historical_20251123_161026.csv

Fetching daily data for MES=F from 2019-05-06 to 2025-11-23...


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_24904\1500899964.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval='1d', progress=False)


Shape: (1652, 5) (rows: 1652, columns: [('Close', 'MES=F'), ('High', 'MES=F'), ('Low', 'MES=F'), ('Open', 'MES=F'), ('Volume', 'MES=F')])
Price         Close     High      Low     Open  Volume
Ticker        MES=F    MES=F    MES=F    MES=F   MES=F
Date                                                  
2019-05-06  2932.50  2947.50  2883.75  2947.50  159243
2019-05-07  2890.75  2930.75  2862.50  2912.00  246925
2019-05-08  2887.25  2899.25  2868.75  2890.25  254506
2019-05-09  2872.75  2885.50  2836.25  2884.25  340659
2019-05-10  2887.00  2893.00  2826.00  2870.25  396057

Earliest date: 2019-05-06
Latest date: 2025-11-21
Saved to MES=F_daily_historical_20251123_161026.csv

Fetching daily data for MYM=F from 2020-05-18 to 2025-11-23...
Shape: (1391, 5) (rows: 1391, columns: [('Close', 'MYM=F'), ('High', 'MYM=F'), ('Low', 'MYM=F'), ('Open', 'MYM=F'), ('Volume', 'MYM=F')])
Price         Close     High      Low     Open  Volume
Ticker        MYM=F    MYM=F    MYM=F    MYM=F   MYM=F
Date   

C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_24904\1500899964.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, interval='1d', progress=False)


In [5]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# --- Configuration ---
TICKER = "MNQ=F"  # Ticker for Micro E-mini NASDAQ 100 Futures (Continuous Contract)
INTERVAL = "1m"   # 1-minute interval

# --- CRITICAL YFINANCE LIMITATION ---
# yfinance (Yahoo Finance) limits 1-minute data retrieval to the last 7-30 days.
# It is NOT possible to fetch 1-minute data back to the inception (May 2019) in one call.

def fetch_recent_futures_data(ticker, interval, period="7d"):
    """
    Fetches the last 7 days of 1-minute futures data, which is the maximum
    reliable period supported by yfinance for this high granularity.
    
    Args:
        ticker (str): The futures contract ticker (e.g., 'MNQ=F').
        interval (str): The desired data interval (e.g., '1m').
        period (str): The lookback period (e.g., '7d', '1mo').
        
    Returns:
        pd.DataFrame: A DataFrame containing the historical data.
    """
    print(f"--- Fetching {period} of {interval} data for {ticker} ---")
    
    try:
        # Use 'period' instead of 'start'/'end' for the most reliable recent data retrieval
        data = yf.download(
            tickers=ticker, 
            interval=interval, 
            period=period,
            # include_prepost=True can sometimes yield more data but is optional
            progress=True 
        )
        
        if data.empty:
            print(f"\n[Warning] No data returned for {ticker} in the last {period}. Ticker or period may be invalid.")
            return None
        
        print("\nSuccessfully fetched data:")
        print(data.head())
        print(f"\nTotal rows fetched: {len(data)}")
        print(f"Data range: {data.index.min()} to {data.index.max()}")
        return data

    except Exception as e:
        print(f"\n[Error] Failed to fetch data: {e}")
        return None

# 1. Fetch the maximum reliable 1-minute history (approx. 7 days)
futures_data = fetch_recent_futures_data(TICKER, INTERVAL)

# --- Demonstrating the Inception Request (Will Fail/Truncate) ---

# Micro E-mini futures (MNQ) inception date was May 2019.
# We set the start date to a day before.
INCEPTION_START_DATE = "2019-05-01"
TODAY = datetime.now().strftime('%Y-%m-%d')

print("\n" + "="*50)
print(f"ATTEMPTING INCEPTION-TO-DATE FETCH: {INCEPTION_START_DATE} to {TODAY}")
print("NOTE: This will likely fail or only return the last 7 days due to Yahoo limits.")
print("="*50)

try:
    # This call is included to demonstrate the user's requested parameters,
    # but with the explicit warning that the results will be truncated by the API.
    full_history_attempt = yf.download(
        tickers=TICKER,
        interval=INTERVAL,
        start=INCEPTION_START_DATE,
        end=TODAY,
        progress=True
    )
    
    print("\n--- Inception Attempt Results (Inspect the Date Range) ---")
    if not full_history_attempt.empty:
        print(f"Attempted to fetch data since: {INCEPTION_START_DATE}")
        print(f"Actual data start date: {full_history_attempt.index.min()}")
        print(f"Actual data end date: {full_history_attempt.index.max()}")
        print(f"Total rows fetched: {len(full_history_attempt)}")
        # Verify if the start date is near the limit (e.g., last 7 days)
        if full_history_attempt.index.min().date() < (datetime.now() - timedelta(days=30)).date():
             print("\n[Result] Unexpectedly large dataset returned. Use with caution.")
        else:
             print("\n[Result] As predicted, data was truncated to the last 7-30 days.")
    else:
        print("Inception fetch returned no data.")

except Exception as e:
    print(f"\n[Error] Inception fetch failed: {e}")

# Example of how to save the reliable recent data
if futures_data is not None:
    filename = f"{TICKER.replace('=F', '')}_1m_recent.csv"
    futures_data.to_csv(filename)
    print(f"\n--- Data saved to {filename} ---")

--- Fetching 7d of 1m data for MNQ=F ---


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_24904\1600252952.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed
C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_24904\1600252952.py:70: FutureWarning: YF.download() has changed argument auto_adjust default to True
  full_history_attempt = yf.download(
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MNQ=F']: YFPricesMissingError('possibly delisted; no price data found  (1m 2019-05-01 -> 2025-11-23) (Yahoo error = "1m data not available for startTime=1556856000 and endTime=1763874000. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")')



Successfully fetched data:
Price                         Close      High       Low      Open Volume
Ticker                        MNQ=F     MNQ=F     MNQ=F     MNQ=F  MNQ=F
Datetime                                                                
2025-11-16 23:10:00+00:00  25085.00  25086.00  25056.50  25073.25      0
2025-11-16 23:11:00+00:00  25093.50  25098.00  25083.25  25084.25   1283
2025-11-16 23:12:00+00:00  25097.25  25104.75  25087.50  25092.50   1223
2025-11-16 23:13:00+00:00  25101.75  25103.50  25091.75  25096.25    713
2025-11-16 23:14:00+00:00  25099.00  25103.25  25098.25  25101.00    485

Total rows fetched: 6849
Data range: 2025-11-16 23:10:00+00:00 to 2025-11-21 21:59:00+00:00

ATTEMPTING INCEPTION-TO-DATE FETCH: 2019-05-01 to 2025-11-23
NOTE: This will likely fail or only return the last 7 days due to Yahoo limits.

--- Inception Attempt Results (Inspect the Date Range) ---
Inception fetch returned no data.

--- Data saved to MNQ_1m_recent.csv ---
